In [ ]:
#checking we are working on the right virtual environment

In [ ]:
import sys
sys.executable
#!pip3 freeze > requirements.txt
#!python --version

# IMPORT PACKAGES

In [ ]:
import numpy as np
import pandas as pd

from ydata_profiling import ProfileReport

import matplotlib.pyplot as plt
import seaborn as sns

from itertools import combinations
from scipy.stats import spearmanr

from sklearn.model_selection import train_test_split
#from sklearn.model_selection import StratifiedShuffleSplit
from math import log2

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

import datetime

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

from yellowbrick.regressor import PredictionError, ResidualsPlot
import shap #numba provides some warnings

from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.preprocessing import PolynomialFeatures

from joblib import dump, load

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor

# DEFING SOME USEFUL FUNCTIONS

In [ ]:
def visualizer():
    viz = PredictionError(model, bestfit=False, alpha=0.2) #it gives problems with bestfit=True
    viz.fit(X_train, y_train)
    viz.score(X_test, y_test)
    scatter_kwargs = {'color': (222/255, 151/255, 228/255)}
    viz.show(bgcolor='k', scatter_kwargs=scatter_kwargs)
    #with ResidualsPlot sometimes kernel dies
    #viz = ResidualsPlot(model, test_alpha=0.1) #, hist=False, qqplot=True
    #viz.fit(X_train, y_train)
    #viz.score(X_test, y_test)
    #viz.show()
    
def visualizer_ct():
    viz = PredictionError(model, bestfit=False, alpha=0.2) #it gives problems with bestfit=True
    viz.fit(X_train_ct, y_train)
    viz.score(X_test_ct, y_test)
    scatter_kwargs = {'color': (222/255, 151/255, 228/255)}
    viz.show(bgcolor='k', scatter_kwargs=scatter_kwargs)

In [ ]:
def metrics():
    print('R2:')
    print('train', r2_score(y_train, model.predict(X_train)))
    print('test:', r2_score(y_test, model.predict(X_test)))
    print('\n')
    print('RMSE')
    print('train', np.sqrt(mean_squared_error(y_train, model.predict(X_train))))
    print('test:', np.sqrt(mean_squared_error(y_test, model.predict(X_test))))
    print('\n')
    print('MAPE:')
    print('train', mean_absolute_percentage_error(y_train, model.predict(X_train))) #Note here that the output is not a percentage in the range [0, 100] and a value of 100 does not mean 100% but 1e2.
    print('test:', mean_absolute_percentage_error(y_test, model.predict(X_test))) #Note here that the output is not a percentage in the range [0, 100] and a value of 100 does not mean 100% but 1e2.
    print('\n')
    
def metrics_ct():
    print('R2:')
    print('train', r2_score(y_train, model.predict(X_train_ct)))
    print('test:', r2_score(y_test, model.predict(X_test_ct)))
    print('\n')
    print('RMSE')
    print('train', np.sqrt(mean_squared_error(y_train, model.predict(X_train_ct))))
    print('test:', np.sqrt(mean_squared_error(y_test, model.predict(X_test_ct))))
    print('\n')
    print('MAPE:')
    print('train', mean_absolute_percentage_error(y_train, model.predict(X_train_ct))) #Note here that the output is not a percentage in the range [0, 100] and a value of 100 does not mean 100% but 1e2.
    print('test:', mean_absolute_percentage_error(y_test, model.predict(X_test_ct))) #Note here that the output is not a percentage in the range [0, 100] and a value of 100 does not mean 100% but 1e2.
    print('\n')

# IMPORT DATASET

In [ ]:
df = pd.read_csv('./Project datasets-20230327/flight_dataset.csv', index_col=0)
df.head()

resetting index, just to be sure

In [ ]:
df.reset_index(inplace=True, drop=True)

# EDA

In [ ]:
#profile = ProfileReport(df, title="FLIGHTS - Pandas Profiling Report", explorative=True)
#profile.to_file("flights.html")

checking for nans

In [ ]:
df.shape

In [ ]:
df.isna().sum()

checking for duplicates

In [ ]:
df.duplicated().sum()

In [ ]:
df.describe()

In [ ]:
########
########

## PRICE

looking at price distribution

In [ ]:
fig, axs = plt.subplots(figsize=(12,10))
sns.histplot(data=df, x='price', kde=True, color='purple')
plt.title('PRICE HISTOGRAM')

price seems to be pretty skewed

we'll probably have to work on it later

In [ ]:
fig, axs = plt.subplots(figsize=(12,10), dpi=300)
sns.boxplot(data=df, x='price', color='violet')
plt.title('PRICE BOXPLOT')

In [ ]:
#########
########

### CITIES

price boxplots on source and destionation cities

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(12,10))

g1 = sns.boxplot(data=df, x='price', y='source_city', ax=axs[0])
g1.set(title = 'BOXPLOT PRICE - SOURCE CITY')

g2 = sns.boxplot(data=df, x='price', y='destination_city', ax=axs[1])
g2.set(title = 'BOXPLOT PRICE - DESTINATION CITY')

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(12,10))

g1 = sns.histplot(data=df, x='price', hue='source_city', kde=True, ax=axs[0])
g1.set(title = 'COUNTPLOT & KDES PRICE - SOURCE CITY')

g2 = sns.histplot(data=df, x='price', hue='destination_city', kde=True, ax=axs[1])
g2.set(title = 'COUNTPLOT & KDES PRICE - DESTINATION CITY')

Indipendently from source or destination city, more economy flights are sold than business flights

In [ ]:
city_cross = pd.crosstab(df['source_city'], df['destination_city'])
city_cross

In [ ]:
for i in city_cross.columns:
    city_cross[f'{i}'].plot(kind='barh')
    #plt.grid()
    plt.title(f'HOW MANY FLIGHTS ARRIVE AT {i}')
    plt.show()

In [ ]:
########
########

### STOPS

price boxplots on the number of stops made

In [ ]:
plt.subplots(figsize=(12,5))
sns.boxplot(data=df, x='price', y='stops')
plt.title('BOXPLOT PRICE - STOPS')

interestingly zero stops flights tend to be the more economic ones, this may be due to the fact that they make smaller voyages


two or more stops have a lot of outliers, may that be last minute flights?

In [ ]:
plt.subplots(figsize=(12,5))
sns.histplot(data=df, x='price', hue='stops', kde=True)
plt.title('COUNTPLOT & KDES PRICE - STOPS')

two or more stops flights are the least sold.

one stop flights sell a lot more than the other two categories and sell for both economy and business trips.

zero stops flights sell mostly economy flights

In [ ]:
stops_cross = pd.crosstab(df['destination_city'], df['stops'])
stops_cross

In [ ]:
for i in stops_cross.columns:
    stops_cross[f'{i}'].plot(kind='barh')
    #plt.grid()
    plt.title(f'HOW MANY FLIGHTS {i} STOP')
    plt.show()

Dheli doen't work a lot with two or more stops flights, but works a lot more than the others with zero stop fligths

In [ ]:
#########
#########

### ARRIVAL TIME

price boxplots on the arrival time

In [ ]:
plt.subplots(figsize=(12,5))
sns.boxplot(data=df, x='price', y='arrival_time')
plt.title('BOXPLOT PRICE - ARRIVAL TIME')

as expected the most economic flights are the one that arrive at destination in the late night or early in the morining

also afternoon flights is reasonable that are a little cheaper than the reaming arrival time

In [ ]:
plt.subplots(figsize=(12,5))
sns.histplot(data=df, x='price', hue='arrival_time', kde=True)
plt.title('COUNTPLOT & KDES PRICE - ARRIVAL TIME')

most of tickets is sold at a low price. Tickets indeferently from the arrival_time, except for early_morning flights, seems to have a little peak at 40.000-60.000 price. That's probably due to business flights sales, while the first peak is for economy sales

In [ ]:
arr_cross = pd.crosstab(df['destination_city'], df['arrival_time'])
arr_cross

In [ ]:
for i in arr_cross.columns:
    arr_cross[f'{i}'].plot(kind='barh')
    #plt.grid()
    plt.title(f'HOW MANY FLIGHTS IN THE {i} ')
    plt.show()

it's interesting to notice that Dheli seems to work a lot in the Late_Night, but pretty much nothing in the Early_Morning

In [ ]:
########
########

### CLASS

price boxplots on class

In [ ]:
plt.subplots(figsize=(12,5), dpi=300)
sns.boxplot(data=df, x='price', y='class')
plt.title('BOXPLOT PRICE - CLASS')

well.. as exepcted economy flights are cheaper

plotting countplot and kdes for economy price and business price

In [ ]:
plt.subplots(figsize=(12,5), dpi=300)
sns.histplot(data=df, x='price', hue='class', kde=True)
plt.title('COUNTPLOT & KDES PRICE - CLASS')

business pricess seems to be pretty much Normally distriubuted, but we can't say the same for economy prices

In [ ]:
########
########

### DURATION

price - duration scatterplot

In [ ]:
sns.scatterplot(data=df, x='price', y='duration')
plt.title('SCATTERPLOT PRICE - DURATION')

paradoxaly longer duration flights are not the more expensive

In [ ]:
#########
#########

## CORRELATION

correlation matrix

In [ ]:
sns.heatmap(df.corr(), annot=True, cmap="coolwarm")

linear correlation is really low, so we expect that linear models won't perform that good.

we still try to predict with a linear model, and then we will compare it's error metric with the one of the other models

Spearman correlation coefficient for categorical variables

In [ ]:
df_cat = df.copy()
for i in df_cat.columns:
    if df_cat[i].dtypes == 'int64' or df_cat[i].dtypes == 'float64':       
           df_cat = df_cat.drop(i, axis=1)
            
for i in df_cat.columns:
    df_cat[i] = df_cat[i].astype('category')

print('Spearman correlation coefficient: \n')
for col1, col2 in combinations(df_cat.columns, 2):
    correlation, p_value = spearmanr(df_cat[col1].cat.codes, df_cat[col2].cat.codes)
    print(col1, '-', col2, ': ', correlation)

In [ ]:
#############
#############

# PRE-PROCESSING

we've already seen that there are non missing values and no duplicates, so we will look for outliers

## OUTLIER DETECTION WITH BOXPLOTS

In [ ]:
fig, axs = plt.subplots(figsize=(12,10), dpi=300)
sns.boxplot(data=df, x='price', color='violet')
plt.title('PRICE BOXPLOT')

calculating interquantile range and conisdering outlier each observation that is over the treshold of +/-1.5*IQR

In [ ]:
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1

out = df[(df['price'] < Q1 - 1.5*IQR) | (df['price'] > Q3 + 1.5*IQR)]
print(len(out), 'outliers were found')

removing outliers from the original df and definig a new one

In [ ]:
not_out = df.loc[~df.index.isin(out.index)]
not_out.head()

In [ ]:
#subsets make sense
not_out.shape[0]+out.shape[0] == df.shape[0]

In [ ]:
fig, axs = plt.subplots(figsize=(12,10))
sns.boxplot(data=not_out, x='price', color='violet')
plt.title('PRICE BOXPLOT - NO OUTLIERS')

we subscribe the original df with our new df without outliers

In [ ]:
df = not_out.copy()

In [ ]:
##########
##########

## STRATIFIED TRAIN-TEST SPLIT

In [ ]:
X, y = df.drop("price", axis=1), df["price"].values.flatten()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=df['class'], random_state=17)

In [ ]:
#########
########

## COLUMN TRANSFORMER

In [ ]:
power = PowerTransformer(method='box-cox')
y_train_bc = power.fit_transform(y_train.reshape(-1, 1))
y_test_bc = power.transform(y_test.reshape(-1, 1))

plotting the transformed target variable

In [ ]:
ytrbc = pd.DataFrame(y_train_bc, columns=['ytrain_boxcox'])

fig, axs = plt.subplots(figsize=(12,10), dpi=300)
sns.histplot(data=ytrbc, x='ytrain_boxcox', kde=True) #, color=(222/255, 151/255, 228/255)
#axs.set_facecolor((0,0,0))
plt.title('PRICE DENSITY DISTRIBUTION - TRANSFORMED') #, color='white'
plt.xlabel('Price') #, color='white'
plt.ylabel('Count') #, color='white'
plt.tick_params() #colors='white'
plt.grid(alpha=0.2)
#fig.patch.set_facecolor((0, 0, 0))
plt.show()

In [ ]:
ct = ColumnTransformer([
    ('hot', OneHotEncoder(drop='first', sparse_output=False), ['airline', 'source_city', 'departure_time', 'arrival_time', 'destination_city', 'class']), 
    ('ord', OrdinalEncoder(categories=[['zero', 'one', 'two_or_more']]), ['stops']),
    ('pow', PowerTransformer(method='yeo-johnson'), ['duration']),
    ('std', StandardScaler(), ['days_left']),
    ('drop', 'drop', ['flight'])
], remainder='passthrough') #passthrough is not useful this time, I'm just used to use it and I think it's good practice
ct
#30 features in output

In [ ]:
class CyclicEncoder(BaseEstimator, TransformerMixin):
    """
    Encode a cyclic feature
    """

    def __init__(self, max_val):
        self.max_val = max_val

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_ = np.empty((len(X), 2))
        X_[:, 0] = np.sin(2 * np.pi * X[:, 0] / self.max_val) #max_val is the value of unique categories in the feature
        X_[:, 1] = np.cos(2 * np.pi * X[:, 0] / self.max_val)
        return X_
    
    def get_feature_names_out(self, input_features=None):
        # Check that input_features contains only one feature
        if len(input_features) != 1:
            raise ValueError("Input array should have exactly one column.")
        
        feature = input_features[0]
        return np.array([f'{feature}_sin', f'{feature}_cos'])

    
cyclical_features = [
    ('departure_time', X_train.departure_time.nunique()),
    ('arrival_time', X_train.arrival_time.nunique())
]


cyclical_transformers = [
    (f"cyclical_{column}", 
     Pipeline(steps=[
         ('ordinal', OrdinalEncoder(categories=[['Early_Morning', 'Morning', 'Afternoon', 'Evening', 'Night', 'Late_Night']])), #we first need to order the categories
         ('cyclical', CyclicEncoder(max_val))
     ]), 
     [column])
    for column, max_val in cyclical_features
]


ctc = ColumnTransformer([
    ('hot', OneHotEncoder(drop='first', sparse_output=False), ['airline', 'source_city', 'destination_city', 'class']), 
    ('ord', OrdinalEncoder(categories=[['zero', 'one', 'two_or_more']]), ['stops']),
    ('pow', PowerTransformer(method='yeo-johnson'), ['duration']),
    ('std', StandardScaler(), ['days_left']),
    ('drop', 'drop', ['flight'])
] + cyclical_transformers,
    remainder='passthrough') #passthrough is not useful this time, I'm just used to use it and I think it's good practice

In [ ]:
########
########

### CORRELATION AND MUTUAL INFO REGRESSION AFTER PRE-PROCESSING WITH DIFFERENT COLUMN TRANSFORMERS

#### CT

##### CORRELATION

In [ ]:
df_corr = ct.fit_transform(df.iloc[:,:-1])
df_corr = pd.DataFrame(df_corr, columns=ct.get_feature_names_out())

y_corr = df.price
y_corr = pd.DataFrame(y_corr, columns=['price'])

df_corr = pd.concat([df_corr, y_corr], axis=1)

sns.set(font_scale=0.7)
plt.subplots(figsize=(25,5), dpi=300)
sns.heatmap(pd.DataFrame(df_corr.corrwith(df_corr['price']), columns=['price']).T, annot=True, cmap='coolwarm', square=True)
plt.title('PRICE CORRELATION - PREPROCESSED DATA\nCT')

##### MUTUAL INFO REGRESSION

In [ ]:
df_corr = ct.fit_transform(df.iloc[:,:-1])
df_corr = pd.DataFrame(df_corr, columns=ct.get_feature_names_out())

#we declare another power transformer, because if we use the same we use on y_train we would then obtain wrong predictions with the inverse_transform method
#power_corr = PowerTransformer() 
y_corr = df.price
y_corr = pd.DataFrame(y_corr, columns=['price'])

mi = mutual_info_regression(df_corr, y_corr)
mi = pd.DataFrame(mi, columns=['price'], index=df_corr.columns)

fig, ax = plt.subplots(figsize=(25,5), dpi=300)
sns.heatmap(mi.T, annot=True, cmap='coolwarm', square=True)
plt.title('PRICE MUTUAL INFO REGRESSION - PREPROCESSED DATA\nCT')
plt.show()

In [ ]:
########
########

#### CTC

##### CORRELATION

In [ ]:
df_corr = ctc.fit_transform(df.iloc[:,:-1])
df_corr = pd.DataFrame(df_corr, columns=ctc.get_feature_names_out())

y_corr = df.price
y_corr = pd.DataFrame(y_corr, columns=['price'])

df_corr = pd.concat([df_corr, y_corr], axis=1)

sns.set(font_scale=0.7)
plt.subplots(figsize=(25,5), dpi=300)
sns.heatmap(pd.DataFrame(df_corr.corrwith(df_corr['price']), columns=['price']).T, annot=True, cmap='coolwarm', square=True)
plt.title('PRICE CORRELATION - PREPROCESSED DATA\nCTC')

##### MUTUAL INFO REGRESSION

In [ ]:
df_corr = ctc.fit_transform(df.iloc[:,:-1])
df_corr = pd.DataFrame(df_corr, columns=ctc.get_feature_names_out())

#we declare another power transformer, because if we use the same we use on y_train we would then obtain wrong predictions with the inverse_transform method
#power_corr = PowerTransformer() 
y_corr = df.price
y_corr = pd.DataFrame(y_corr, columns=['price'])

mi = mutual_info_regression(df_corr, y_corr)
mi = pd.DataFrame(mi, columns=['price'], index=df_corr.columns)

fig, ax = plt.subplots(figsize=(25,5), dpi=300)
sns.heatmap(mi.T, annot=True, cmap='coolwarm', square=True)
plt.title('PRICE MUTUAL INFO REGRESSION - PREPROCESSED DATA\nCTC')
plt.show()

In [ ]:
#########
#########

# MODEL 1 - LINEAR MODEL

## LINEAR REGRESSION

In [ ]:
model = LinearRegression()
poly = PolynomialFeatures()
pipe = Pipeline([
    ('column_transformer', ct),
    ('polynomial', poly),
    ('regressor', model)
])
pipe

In [ ]:
param_grid = {
    'column_transformer': [ct, ctc],
    'polynomial__degree': [1, 2, 3]
}

adesso = datetime.datetime.now()
###
grid = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    cv=3,
    verbose=3, 
    n_jobs=-1, 
    pre_dispatch=4,
)

model_grid = grid.fit(X_train, y_train.ravel())
###
adesso_2 = datetime.datetime.now()
tempo_elapsed = adesso_2 - adesso
tempo_elapsed

In [ ]:
model_grid.best_params_

In [ ]:
model = model_grid.best_estimator_
dump(model, 'flights_model/flights_linear_poly.pkl')
metrics()

In [ ]:
visualizer()

In [ ]:
########

lets try with leave one out instead of one hot encoder

In [ ]:
model = LinearRegression()
#rfecv = RFECV(estimator=model, cv=3)
kbest = SelectKBest(score_func=mutual_info_regression)
poly = PolynomialFeatures()
pipel = Pipeline([
    ('column_transformer', ctl),
    #('selection', rfecv),
    ('selection', kbest),
    ('polynomial', poly),
    ('regressor', model)
])
pipel

In [ ]:
param_grid = {
    #'selection__min_features_to_select': [10, 8, 5],
    'selection__k': [10, 8, 5, 3, 1],
    'polynomial__degree': [1, 2, 3],
    'regressor__fit_intercept': [True]
}

adesso = datetime.datetime.now()
###
#scoring = ['accuracy','recall']
grid = GridSearchCV(
    estimator=pipel,
    param_grid=param_grid,
    cv=3,
    verbose=3, #quando usiamo il parallel computer (n_jobs) il verbose non funziona
    #n_jobs=-1, #usa tutti i kernel 
    #pre_dispatch=4,
    #scoring=scoring
)

model_grid = grid.fit(X_train, y_train)
###
adesso_2 = datetime.datetime.now()
tempo_elapsed = adesso_2 - adesso
tempo_elapsed

In [ ]:
model_grid.best_params_

In [ ]:
model = model_grid.best_estimator_
metrics()

In [ ]:
visualizer()

In [ ]:
preds = pd.DataFrame(power.inverse_transform(y_test), columns=['y_pred'])
sns.histplot(data=preds, x='y_pred', kde=True)
plt.title('LINEAR REGRESSION - PREDICTIONS')

In [ ]:
#########
#########

# MODEL 2 - ENSAMBLE MODELS

## RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestRegressor

### NO TUNE TRY

In [ ]:
X_train_ct = ctc.fit_transform(X_train)
X_test_ct = ctc.transform(X_test)

In [ ]:
model = RandomForestRegressor()
model.fit(X_train_ct, y_train.ravel())

In [ ]:
#dump(model, '###.pkl')
#dump(power, 'flights_test_power_scaler.pkl')
metrics_ct()

In [ ]:
visualizer_ct()

In [ ]:
preds = pd.DataFrame(power.inverse_transform(y_test), columns=['y_pred'])
sns.histplot(data=preds, x='y_pred', kde=True)
plt.title('RANDOM FOREST - PREDICTIONS')

### TUNING

In [ ]:
model = RandomForestRegressor()
pipe = Pipeline([
    ('column_transformer', ctc),
    ('regressor', model)
])
pipe

In [ ]:
#valutare di fare la random grid search, che 2025 non sono pochissimissimi
param_grid = {
    'regressor__n_estimators': [500, 1000], #100, 200, #, 5000 it's the best one, but takes about 50 minutes per model
    'regressor__max_depth': [None], #10, 20, 30, 
    'regressor__min_samples_split': [3, 5, 7], #2, #, 10
    'regressor__min_samples_leaf': [1], #, 2, 4
    'regressor__max_features': [1.0] #, 'sqrt', 'log2'
}

adesso = datetime.datetime.now()
###
#scoring = ['accuracy','recall']
grid = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    cv=3,
    verbose=3, #quando usiamo il parallel computer (n_jobs) il verbose non funziona
    n_jobs=-1, #usa tutti i kernel 
    pre_dispatch=4,
    #scoring=scoring,
    return_train_score=True
)

model_grid = grid.fit(X_train, y_train.ravel())
###
adesso_2 = datetime.datetime.now()
tempo_elapsed = adesso_2 - adesso
tempo_elapsed

In [ ]:
model_grid.best_params_

In [ ]:
model = model_grid.best_estimator_
dump(model, 'flights_rf.pkl')
dump(ctc, 'flights_cyc_ct.pkl')
dump(power, 'flights_test_power_scaler.pkl')
metrics()

In [ ]:
visualizer()

In [ ]:
##########
#########

### FEATURE IMPORTANCE - SHAP

In [ ]:
model = load('flights_rf.pkl')

In [ ]:
#it doesn't seem to be able to compute shap values for this specific model

In [ ]:
shap.initjs()

X_train_ct = pd.DataFrame(model['column_transformer'].transform(X_train), columns=model['column_transformer'].get_feature_names_out())
s_X_train_ct = X_train_ct.sample(n=1000) #, random_state=42
model_reg = model['regressor']

explainer = shap.Explainer(model_reg)
shap_values = explainer(s_X_train_ct)

# Get the absolute SHAP values for each feature
abs_shap_values = np.abs(shap_values.values).mean(0)

In [ ]:
shap.summary_plot(shap_values, s_X_train_ct)

In [ ]:
shap.plots.bar(shap_values)

In [ ]:
shap.force_plot(explainer.expected_value, shap_values.values[0,:], s_X_train_ct.iloc[0,:])

In [ ]:
#feature_names = list(model['column_transformer'].get_feature_names_out())
#
#features_values = pd.DataFrame(feature_names, columns=['features'])
#features_values['abs_shap_values'] = list(abs_shap_values)
#
#to_drop = list(features_values.query('abs_shap_values == 0').features)

In [ ]:
#########
#########

## HIST GRADIENT BOOSTING

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor

### NO TUNE TRY

In [ ]:
X_train_ct = ctc.fit_transform(X_train)
X_test_ct = ctc.transform(X_test)

In [ ]:
#weights = []
#minimo = -2.7 #min(y_train)
#for i in y_train:
#    if i == minimo:
#        weights.append(2)
#    else:
#        weights.append(1)

In [ ]:
model = HistGradientBoostingRegressor()
model.fit(X_train_ct, y_train.ravel()) #,  sample_weight=weights

In [ ]:
#dump(model, 'hist_regressor.pkl')
#dump(power, 'flights_test_power_scaler.pkl')
metrics_ct()

In [ ]:
visualizer_ct()

In [ ]:
#preds = pd.DataFrame(power.inverse_transform(y_test), columns=['y_pred'])
#sns.histplot(data=preds, x='y_pred', kde=True)
#plt.title('HIST GRADIENT BOOSTING - PREDICTIONS')

In [ ]:
#########
#########

### FEATURE SELECTION - SHAP
#questa poi è da togliere visto che la feature selection la facciamo poi sul modello migliore della grid search

we arleady knwew about shap and what it does, but we've taken code from chat gpt to actually use it

In [ ]:
shap.initjs()

feature_names = ctc.get_feature_names_out()

explainer = shap.Explainer(model)
shap_values = explainer(X_test_ct) #capire se si deve usare X_train o X_test

# Get the absolute SHAP values for each feature
abs_shap_values = np.abs(shap_values.values).mean(0)

# Sort the features by average absolute SHAP value
importance_order = np.argsort(abs_shap_values)[::-1]

X_test_ct = pd.DataFrame(X_test_ct, columns=feature_names) #capire se si deve usare X_train o X_test

# Print the feature importance ranking
print("Feature importance ranking:")
for i in importance_order:
    print(X_test_ct.columns[i], abs_shap_values[i])

In this chart, the x-axis stands for SHAP value, and the y-axis has all the features. Each point on the chart is one SHAP value for a prediction and feature. Red color means higher value of a feature. Blue means lower value of a feature. We can get the general sense of features’ directionality impact based on the distribution of the red and blue dots.

https://m.mage.ai/how-to-interpret-and-explain-your-machine-learning-models-using-shap-values-471c2635b78e

In [ ]:
shap.summary_plot(shap_values, X_train_ct)

What Shapley does is quantifying the contribution that each player brings to the game. What SHAP does is quantifying the contribution that each feature brings to the prediction made by the model.

https://towardsdatascience.com/shap-explained-the-way-i-wish-someone-explained-it-to-me-ab81cc69ef30

In [ ]:
shap_values.feature_names = feature_names
shap.plots.bar(shap_values)

features pushing the prediction higher are shown in red ..., those pushing the prediction lower are in blue...

https://datascience.stackexchange.com/questions/65502/how-to-interpret-shapley-force-plot-for-feature-importance

In [ ]:
#this refers to the first row of x_train_ct
shap.force_plot(explainer.expected_value, shap_values.values[0,:], X_train_ct.iloc[0,:])

In [ ]:
X_train_ct.columns

In [ ]:
##########
##########

### TUNING

In [ ]:
model = HistGradientBoostingRegressor()
pipe = Pipeline([
    ('column_transformer', ct), #ctc
    ('regressor', model)
])
pipe

In [ ]:
#valutare di fare la random grid search, che 2025 non sono pochissimissimi
param_grid = {
    'column_transformer': [ct, ctl, ctc],
    'regressor__max_iter': [1500], #100, 200, #500, #750, #1000, #1250, #, 1750, 2000
    'regressor__learning_rate': [0.3], #, 0.01, 0.001 #0.1, #, 0.5 #, 0.4
    'regressor__max_depth': [50], # 5, 10, #None, 20, #30, #40, #, 60
    'regressor__min_samples_leaf': [10], #1, , 20
    'regressor__l2_regularization': [0.2, 0.3], #0.0, , 1.0 #, 0.3 #0.1, #, 0.4
    'regressor__loss': ['squared_error'], #, 'absolute_error'
    'regressor__max_leaf_nodes': [40, 50, 60], #None, 10, 20, 
    'regressor__max_bins': [255], #2, 50, 100, 150, 200, 
    #'regressor__early_stopping': [False, True]
}

adesso = datetime.datetime.now()
###
#scoring = ['accuracy','recall']
grid = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    cv=3,
    verbose=3, #quando usiamo il parallel computer (n_jobs) il verbose non funziona
    n_jobs=-1, #usa tutti i kernel 
    #pre_dispatch=4,
    #scoring=scoring,
    return_train_score=True
)

model_grid = grid.fit(X_train, y_train.ravel())
###
adesso_2 = datetime.datetime.now()
tempo_elapsed = adesso_2 - adesso
tempo_elapsed

In [ ]:
model_grid.best_params_

In [ ]:
model = model_grid.best_estimator_
dump(model, 'flights_histgbr.pkl')
dump(power, 'flights_test_power_scaler.pkl')
metrics()

In [ ]:
visualizer()

In [ ]:
preds = pd.DataFrame(power.inverse_transform(y_test), columns=['y_pred'])
sns.histplot(data=preds, x='y_pred', kde=True)
plt.title('HIST GRADIENT BOOSTING REGRESSOR - PREDICTIONS')

In [ ]:
##########
##########

### FEATRUE SELECTION POST GRID SEARCH

In [ ]:
model = load('flights_histgbr.pkl')

In [ ]:
#it doesn't seem to be able to compute shap values for this specific model

In [ ]:
shap.initjs()

X_train_ct = pd.DataFrame(model['column_transformer'].transform(X_train), columns=model['column_transformer'].get_feature_names_out())
s_X_train_ct = X_train_ct.sample(n=1000) #, random_state=42
model_reg = model['regressor']

explainer = shap.Explainer(model_reg)
shap_values = explainer(s_X_train_ct)

# Get the absolute SHAP values for each feature
abs_shap_values = np.abs(shap_values.values).mean(0)

In [ ]:
shap.summary_plot(shap_values, s_X_train_ct)

In [ ]:
shap.plots.bar(shap_values)

In [ ]:
shap.force_plot(explainer.expected_value, shap_values.values[0,:], s_X_train_ct.iloc[0,:])

In [ ]:
#selezionare feature non importanti

In [ ]:
abs_shap_values.sort()
abs_shap_values.tolist()

In [ ]:
############
###########

### TUNING WITH OPTUNA

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import cross_val_score
import optuna

In [ ]:
def objective(trial):
    param = {
        'max_iter': trial.suggest_int('max_iter', 750, 1000),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 20, 40),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 10, 30),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.35),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'l2_regularization': trial.suggest_float('l2_regularization', 0.01, 0.3),
        'loss': trial.suggest_categorical('loss', ['squared_error', 'absolute_error'])
    }
    
    model = HistGradientBoostingRegressor(**param)
    return cross_val_score(model, X_train_ct, y_train.ravel(), cv=3, n_jobs=-1, scoring='r2').mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print('Best trial: score {}, params {}'.format(study.best_trial.value, study.best_trial.params))

In [ ]:
from optuna.visualization import plot_optimization_history

plot_optimization_history(study)

In [ ]:
from optuna.visualization import plot_param_importances

plot_param_importances(study)

In [ ]:
optuna_params = study.best_trial.params
#xgb.train(optuna_params, dtrain, num_boost_round=300)

In [ ]:
##########
##########

## GRADIENT DESCEND BOOSTING

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)

In [ ]:
model = GradientBoostingRegressor()
model.fit(X_train, y_train.ravel())

In [ ]:
#dump(model, '###.pkl')
#dump(power, 'flights_test_power_scaler.pkl')
metrics()

In [ ]:
visualizer()

In [ ]:
preds = pd.DataFrame(power.inverse_transform(y_test), columns=['y_pred'])
sns.histplot(data=preds, x='y_pred', kde=True)
plt.title('GRADIENT DESCEND BOOSTING - PREDICTIONS')

In [ ]:
#########
#########

## ADABOOST

In [ ]:
from sklearn.ensemble import AdaBoostRegressor

In [ ]:
X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)

In [ ]:
model = AdaBoostRegressor()
model.fit(X_train, y_train.ravel())

In [ ]:
#dump(model, '###.pkl')
#dump(power, 'flights_test_power_scaler.pkl')
metrics()

In [ ]:
visualizer()

In [ ]:
preds = pd.DataFrame(power.inverse_transform(y_test), columns=['y_pred'])
sns.histplot(data=preds, x='y_pred', kde=True)
plt.title('ADABOOST - PREDICTIONS')

In [ ]:
#########
#########

## XGBOOST

In [ ]:
from xgboost import XGBRegressor

### NO TUNE TRY

In [ ]:
X_train_ct = ctc.fit_transform(X_train)
X_test_ct = ctc.transform(X_test)

In [ ]:
model = XGBRegressor()
model.fit(X_train_ct, y_train.ravel())

In [ ]:
#dump(model, '###.pkl')
#dump(power, 'flights_test_power_scaler.pkl')
metrics_ct()

In [ ]:
visualizer_ct()

In [ ]:
preds = pd.DataFrame(power.inverse_transform(y_test), columns=['y_pred'])
sns.histplot(data=preds, x='y_pred', kde=True)
plt.title('XGBOOST - PREDICTIONS')

In [ ]:
##########
##########

### TUNING

In [ ]:
model = XGBRegressor()
pipe = Pipeline([
    ('column_transformer', ct), #ctc
    ('regressor', model)
])
pipe

In [ ]:
#from joblib import parallel_backend

In [ ]:
#param_grid = {
#    'column_transformer': [ct, ctl, ctc],
#    'regressor__n_estimators': [100, 200, 500],
#    'regressor__max_depth': [3, 5, 7, 10],
#    'regressor__learning_rate': [0.01, 0.1, 0.2],
#    'regressor__subsample': [0.5, 0.7, 1.0],
#    'regressor__colsample_bytree': [0.4, 0.6, 0.8, 1.0],
#    'regressor__gamma': [0, 0.1, 0.2],
#    'regressor__reg_alpha': [0, 0.5, 1.0],
#    'regressor__reg_lambda': [0.5, 1.0, 1.5]
#}
#
#adesso = datetime.datetime.now()
####
##scoring = ['accuracy','recall']
#grid = GridSearchCV(
#    estimator=pipe,
#    param_grid=param_grid,
#    cv=3,
#    verbose=3, #quando usiamo il parallel computer (n_jobs) il verbose non funziona
#    n_jobs=-1, #usa tutti i kernel 
#    #pre_dispatch=4,
#    #scoring=scoring,
#    return_train_score=True
#)
#
#model_grid = grid.fit(X_train, y_train.ravel())
####
#adesso_2 = datetime.datetime.now()
#tempo_elapsed = adesso_2 - adesso
#tempo_elapsed

In [ ]:
param_grid = {
    'column_transformer': [ct, ctl, ctc],
    'regressor__n_estimators': [500], #, 200 #, 500
    'regressor__max_depth': [1, 6], # 7, 10
    'regressor__learning_rate': [0.01, 0.1],
    #'regressor__subsample': [0.5, 0.7, 1.0],
    'regressor__gamma': [0, 0.1],
    'regressor__reg_lambda': [1.0, 1.5],
    'regressor__num_parallel_tree': [1, 2]
}

adesso = datetime.datetime.now()

grid = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    cv=3,
    verbose=3, #quando usiamo il parallel computer (n_jobs) il verbose non funziona
    n_jobs=-1, #usa tutti i kernel 
    #pre_dispatch=4,
    #scoring=scoring,
    return_train_score=True
)
model_grid = grid.fit(X_train, y_train.ravel())

###
adesso_2 = datetime.datetime.now()
tempo_elapsed = adesso_2 - adesso
tempo_elapsed

In [ ]:
model_grid.best_params_

In [ ]:
model = model_grid.best_estimator_
model['regressor'].save_model('flights_xgb.json')
metrics()

In [ ]:
visualizer()

In [ ]:
#model.save?

In [ ]:
#model.save_model('flights_xgboost.json')

In [ ]:
##########
##########

## XGBOOST DMATRICES

In [ ]:
import xgboost as xgb

### NO TUNE TRY

In [ ]:
#X_train_ct = ct.fit_transform(X_train)
#X_test_ct = ct.transform(X_test)

In [ ]:
X_train_ct = ctc.fit_transform(X_train)
X_test_ct = ctc.transform(X_test)

code was taken from chatgpt

In [ ]:
#dtrain = xgb.DMatrix(X_train_ct, label=y_train)
#dtest = xgb.DMatrix(X_test_ct, label=y_test)

In [ ]:
#########
#########

### LETS TRY OPTUNA

In [ ]:
import optuna
import xgboost as xgb

code taken from chatgpt

In [ ]:
## Define the objective function
#def objective(trial):
#    # Split your data into training and validation sets
#    #X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
#
#    # Convert your data into DMatrix format
#    dtrain = xgb.DMatrix(X_train_ct, label=y_train)
#    dtest = xgb.DMatrix(X_test_ct, label=y_test)
#
#    # Define the hyperparameter configuration space
#    param = {
#        'objective': 'reg:squarederror',
#        'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart']),
#        'lambda': trial.suggest_float('lambda', 1e-8, 1.0),
#        'alpha': trial.suggest_float('alpha', 1e-8, 1.0),
#    }
#
#    if param['booster'] == 'gbtree' or param['booster'] == 'dart':
#        param['max_depth'] = trial.suggest_int('max_depth', 1, 9)
#        param['eta'] = trial.suggest_float('eta', 1e-8, 1.0)
#        param['gamma'] = trial.suggest_float('gamma', 1e-8, 1.0)
#        param['grow_policy'] = trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide'])
#
#    # Train the model
#    bst = xgb.train(param, dtrain)
#    
#    # Predict on the validation set and compute the mean squared error
#    preds = bst.predict(dtest)
#    mse = ((y_test - preds) ** 2).mean()
#    
#    return mse
#
## Run the optimization
#study = optuna.create_study(direction='minimize')
#study.optimize(objective, n_trials=100)
#
## Print the best parameters
#print(study.best_trial.params)

In [ ]:
import optuna
import xgboost as xgb
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score

In [ ]:
#xgb.train?

provare tweedie distribution sennza box_cox

In [ ]:
#bisognerebbe provare a passargli i tre vari column transformer
#sistemare la cosa del train e del valid.
#non utlizza tutto il dataset


# Define the objective function
def objective(trial):
    # Split your data into training and validation sets
    X_train, X_valid, y_train_op, y_valid = train_test_split(X_train_ct, y_train, test_size=0.2) #, random_state=42

    # Convert your data into DMatrix format
    dtrain = xgb.DMatrix(X_train, label=y_train_op)
    dvalid = xgb.DMatrix(X_valid, label=y_valid)

    # Define the hyperparameter configuration space
    param = {
        'objective': 'reg:squarederror',
        #'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart']),
        'booster': trial.suggest_categorical('booster', ['gbtree']),
        'lambda': trial.suggest_float('lambda', 0.3, 1.0), #1e-8
        'alpha': trial.suggest_float('alpha', 0.3, 1.0),
    }

    if param['booster'] == 'gbtree':
        param['max_depth'] = trial.suggest_int('max_depth', 1, 10) #3, 10
        param['eta'] = trial.suggest_float('eta', 0.0001, 0.3) #1.0
        param['gamma'] = trial.suggest_float('gamma', 0.003, 1.0)
        param['grow_policy'] = trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide'])
        param['subsample'] = trial.suggest_float('subsample', 0, 1.0)
        param['tree_method'] = trial.suggest_categorical('tree_method', ['approx', 'hist'])
        #param['num_parallel_tree'] = trial.suggest_int('num_parallel_tree', 100, 101) #, 1000
        param['colsample_bytree'] = trial.suggest_float('colsample_bytree', 0.5, 1.0)
        param['colsample_bylevel'] = trial.suggest_float('colsample_bylevel', 0.5, 1.0)
        param['colsample_bynode'] = trial.suggest_float('colsample_bynode', 0.5, 1.0)
        param['min_child_weight'] = trial.suggest_int('min_child_weight', 0, 10)

    # Train the model
    bst = xgb.train(param, dtrain, num_boost_round=500)
    
    ## Train the model with early stopping
    #bst = xgb.train(param, dtrain, num_boost_round=2000, evals=[(dvalid, "validation")], early_stopping_rounds=50) #2000

    
    # Predict on the validation set and compute the r2 score
    preds = bst.predict(dvalid)
    r2 = r2_score(y_valid, preds)
    
    # Since we want to maximize R-squared, we return it as a negative value
    return r2

# Run the optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1000) #300

# Print the best parameters
print(study.best_trial.params)

In [ ]:
print("Best R-squared:", -study.best_value)

In [ ]:
from optuna.visualization import plot_optimization_history

plot_optimization_history(study)

In [ ]:
from optuna.visualization import plot_param_importances

plot_param_importances(study)

In [ ]:
optuna_params = study.best_trial.params
xgb.train(optuna_params, dtrain, num_boost_round=300)

In [ ]:
##########
##########

In [ ]:
#bisognerebbe provare a passargli i tre vari column transformer
#sistemare la cosa del train e del valid.
#non utlizza tutto il dataset


# Define the objective function
def objective(trial):
    # Split your data into training and validation sets
    X_train, X_valid, y_train_op, y_valid = train_test_split(X_train_ct, y_train, test_size=0.2) #, random_state=42

    # Convert your data into DMatrix format
    dtrain = xgb.DMatrix(X_train, label=y_train_op)
    dvalid = xgb.DMatrix(X_valid, label=y_valid)

    # Define the hyperparameter configuration space
    param = {
        'objective': 'reg:squarederror',
        #'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart']),
        'booster': trial.suggest_categorical('booster', ['gbtree', 'dart']),
        'lambda': trial.suggest_float('lambda', 0.3, 1.0), #1e-8
        'alpha': trial.suggest_float('alpha', 0.3, 1.0),
    }

    if param['booster'] in ['gbtree', 'dart']:
        param['max_depth'] = trial.suggest_int('max_depth', 1, 10)
        param['eta'] = trial.suggest_float('eta', 0.001, 1.0)
        param['gamma'] = trial.suggest_float('gamma', 0.01, 1.0)
        param['grow_policy'] = trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide'])
        param['subsample'] = trial.suggest_float('subsample', 0.5, 1.0)
        param['tree_method'] = trial.suggest_categorical('tree_method', ['auto', 'hist'])
        param['colsample_bytree'] = trial.suggest_float('colsample_bytree', 0.5, 1.0)
        param['min_child_weight'] = trial.suggest_int('min_child_weight', 0, 10)

    # If 'dart', then add 'sample_type', 'normalize_type', and 'rate_drop'
    if param['booster'] == 'dart':
        param['sample_type'] = trial.suggest_categorical('sample_type', ['uniform', 'weighted'])
        param['normalize_type'] = trial.suggest_categorical('normalize_type', ['tree', 'forest'])
        param['rate_drop'] = trial.suggest_float('rate_drop', 0, 1)

    # Train the model
    bst = xgb.train(param, dtrain, num_boost_round=200)
    
    ## Train the model with early stopping
    #bst = xgb.train(param, dtrain, num_boost_round=2000, evals=[(dvalid, "validation")], early_stopping_rounds=50) #2000

    
    # Predict on the validation set and compute the r2 score
    preds = bst.predict(dvalid)
    r2 = r2_score(y_valid, preds)
    
    # Since we want to maximize R-squared, we return it as a negative value
    return r2

# Run the optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1000) #300

# Print the best parameters
print(study.best_trial.params)

In [ ]:
print("Best R-squared:", -study.best_value)

In [ ]:
from optuna.visualization import plot_optimization_history

plot_optimization_history(study)

In [ ]:
from optuna.visualization import plot_param_importances

plot_param_importances(study)

#### PROVOLA MULTI-OBJECTIVE

In [ ]:
import optuna
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [ ]:
# Define the objective function
def objective(trial):
    # Split your data into training and validation sets
    X_train, X_valid, y_train_op, y_valid = train_test_split(X_train_ct, y_train, test_size=0.2) #, random_state=42

    # Convert your data into DMatrix format
    dtrain = xgb.DMatrix(X_train, label=y_train_op)
    dvalid = xgb.DMatrix(X_valid, label=y_valid)

    # Define the hyperparameter configuration space
    param = {
        'objective': 'reg:squarederror',
        'booster': trial.suggest_categorical('booster', ['gbtree']),
        'lambda': trial.suggest_float('lambda', 0, 1.0), 
        'alpha': trial.suggest_float('alpha', 0, 1.0),
    }

    if param['booster'] == 'gbtree':
        param['max_depth'] = trial.suggest_int('max_depth', 1, 10)
        param['eta'] = trial.suggest_float('eta', 0, 1.0)
        param['gamma'] = trial.suggest_float('gamma', 0, 1.0)
        param['grow_policy'] = trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide'])
        param['subsample'] = trial.suggest_float('subsample', 0.5, 1.0)
        param['colsample_bytree'] = trial.suggest_float('colsample_bytree', 0.5, 1.0)
        param['min_child_weight'] = trial.suggest_int('min_child_weight', 0, 10)

    # Train the model
    bst = xgb.train(param, dtrain, num_boost_round=200) 

    # Predict on the validation set
    preds = bst.predict(dvalid)
    
    # Compute the objectives
    mape = mean_absolute_percentage_error(y_valid, preds)
    rmse = np.sqrt(mean_squared_error(y_valid, preds))
    r2 = r2_score(y_valid, preds)
    
    # Return a tuple of the objectives
    return -r2, rmse, mape

# Create a multi-objective study
study = optuna.create_study(directions=['minimize', 'minimize', 'minimize'])

# Optimize the study
study.optimize(objective, n_trials=1000)

# Print the best parameters
print(study.best_trial.params)

In [ ]:
print("Best R-squared:", study.best_trials)

In [ ]:
from optuna.visualization import plot_optimization_history

plot_optimization_history(study)

In [ ]:
from optuna.visualization import plot_param_importances

plot_param_importances(study, target='r2')

In [ ]:
from optuna.visualization import plot_pareto_front

plot_pareto_front(study)

In [ ]:
#######

with crossvalidation

In [ ]:
##bisognerebbe provare a passargli i tre vari column transformer
#
## Define the objective function
#def objective(trial):
#    # Define the hyperparameter configuration space
#    param = {
#        'objective': 'reg:squarederror',
#        'booster': trial.suggest_categorical('booster', ['gbtree']),
#        'lambda': trial.suggest_float('lambda', 1e-8, 1.0),
#        'alpha': trial.suggest_float('alpha', 1e-8, 1.0),
#    }
#
#    if param['booster'] == 'gbtree':
#        param['max_depth'] = trial.suggest_int('max_depth', 1, 10)
#        param['eta'] = trial.suggest_float('eta', 1e-8, 1.0)
#        param['gamma'] = trial.suggest_float('gamma', 1e-8, 1.0)
#        param['grow_policy'] = trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide'])
#        param['subsample'] = trial.suggest_float('subsample', 0.5, 1.0)
#        param['tree_method'] = trial.suggest_categorical('tree_method', ['auto', 'hist'])
#        param['num_parallel_tree'] = trial.suggest_int('max_depth', 1, 5)
#
#    # Create the XGBoost model
#    model = xgb.XGBRegressor(**param)
#
#    # Perform cross-validation and return the mean R2 score
#    score = cross_val_score(model, X_train_ct, y_train, cv=3, scoring='r2').mean()
#
#    # Since we want to maximize R-squared, we return it as a negative value
#    return -score
#
## Run the optimization
#study = optuna.create_study(direction='minimize')
#study.optimize(objective, n_trials=100)
#
## Print the best parameters
#print(study.best_trial.params)

In [ ]:
print("Best R-squared:", -study.best_value)

In [ ]:
#########
#########

# adfgdfhkgjl